In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
plt.interactive(False)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",  one_hot=True)
global_step = tf.Variable(0, trainable=False, name='global_step')

#변수들
X = tf.placeholder(tf.float32,[None, 28, 28, 1], name='X')          # [?, 28, 28, 1]
Y = tf.placeholder(tf.float32,[None, 10], name='Y')                     # [?, 10]
keep_prob = tf.placeholder(tf.float32)
    
"""모델"""
W1 = tf.get_variable("W1", shape=[3, 3, 1, 16], initializer=tf.contrib.layers.xavier_initializer())   
L1 = tf.nn. conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME', name='L1_conv')  # [?, 28, 28, 16]
L1 = tf.nn.relu(L1, name='L1_relu')

W2 = tf.get_variable("W2", shape=[3, 3, 16, 32], initializer=tf.contrib.layers.xavier_initializer())   # [3, 3, 16, 16]
L2 = tf.nn. conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME', name='L2_conv')      # [?, 28, 28, 32]
L2 = tf.nn.relu(L2, name='L2_relu')
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='L2_pool')       # [?,14, 14, 32]

W3 = tf.get_variable("W3", shape=[3, 3, 32, 64], initializer=tf.contrib.layers.xavier_initializer())   # [3, 3, 16, 16]
L3 = tf.nn. conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME', name='L3_conv')  # [?, 14, 14,  64]
L3 = tf.nn.relu(L3, name='L3_relu')

W4 = tf.get_variable("W4", shape=[3, 3, 64, 64], initializer=tf.contrib.layers.xavier_initializer())   # [3, 3, 16, 16]
L4 = tf.nn. conv2d(L3, W4, strides=[1, 1, 1, 1], padding='SAME', name='L4_conv')  # [?, 14, 14,  64]
L4 = tf.nn.relu(L3, name='L4_relu')
L4 = tf.nn.max_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='L4_pool')   # [?, 7, 7,  64]

W5 = tf.get_variable("W5", shape=[7 * 7* 64, 256], initializer=tf.contrib.layers.xavier_initializer())   # [3, 3, 16,16]
L5 = tf.reshape(L4, [-1, 7 * 7 * 64])
L5 = tf. matmul(L5, W5, name='L5_matmul')
L5 = tf.nn.relu(L5, name='L5_relu')
L5 = tf.nn.dropout(L5, keep_prob, name='L5_dropout')

W6 = tf.get_variable("W6", shape=[256, 10], initializer=tf.contrib.layers.xavier_initializer())   # [3, 3, 16, 16]
L6 = tf.matmul(L5, W6, name='L6_matmul')

model = tf.nn.dropout(L6, keep_prob, name='model')

cost =   tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))                        
optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(cost, global_step=global_step)      

#검증을 위한 코드/모니터링
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
tf.summary.scalar('cost', cost)                    
tf.summary.scalar('accuracy', accuracy)    
tf.summary.histogram("model", model)   
#tf.summary.image("L1",L1)

#파라미터
training_epoch = 100
batch_size = 100

#세션런 
with tf.Session() as sess:                                                                                                                   
    sess.run(tf.global_variables_initializer())                                                   
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs/cnn_mnist_vgg_xa', sess.graph)    
    
    for epoch in range(training_epoch):
        avg_cost = 0
        total_batch =  int(mnist.train.num_examples / batch_size)   
       
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(-1, 28, 28, 1)
            c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
            avg_cost += c / total_batch
                            
            summary = sess.run(merged, feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 1})
            writer.add_summary(summary, global_step=sess.run(global_step))
        
        #정확도 출력
        test_xs, test_ys = mnist.test.next_batch(1000)
        test_xs = test_xs.reshape(-1, 28, 28, 1)
        print('Step:', '%03d '%sess.run(global_step),' Cost:', '%.4f'%(avg_cost), ' Acc:', 
              sess.run(accuracy, feed_dict={X:test_xs, Y:test_ys, keep_prob: 1}))
        
    #그림과 레이블 출력
    fig, ax = plt.subplots(1, 10)   
    for i in range(10):
        ax[i].set_xticks([])
        ax[i].set_yticks([])
        test_label = sess.run(model, feed_dict={X:[test_xs[i]], Y:[test_ys[i]], keep_prob: 1})
        ax[i].set_xlabel(np.argmax(test_label))
        ax[i].imshow(test_xs[i].reshape(28,28), cmap='gray')    
    plt.show(block=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please write your own downloading logic.


Instructions for updating:
Please use tf.data to implement this functionality.


Extracting ./mnist/data/train-images-idx3-ubyte.gz


Instructions for updating:
Please use tf.data to implement this functionality.


...

Step: 53350   Cost: 0.4074  Acc: 0.992


Step: 53900   Cost: 0.4069  Acc: 0.992


Step: 54450   Cost: 0.4051  Acc: 0.99


Step: 55000   Cost: 0.4076  Acc: 0.994


<Figure size 640x480 with 10 Axes>